In [1]:
#https://projects.ncsu.edu/crsc/reports/ftp/pdf/crsc-tr06-19.pdf
#https://github.com/lbenz730/world_cup
import pandas as pd
import numpy as np

In [2]:
int_games = pd.read_csv("https://raw.githubusercontent.com/lbenz730/world_cup/master/international_soccer_game_data.csv")

In [3]:
int_games['date'] = pd.to_datetime(int_games['date'])

In [4]:
int_games['recent'] = pd.datetime.now().date() - int_games['date'] 
int_games['GD'] = int_games['home_score'] - int_games['away_score']

In [5]:
pr_alo = True

In [6]:
if not pr_alo:
    home_games = int_games[['home_team','away_team','tournament','neutral','recent','date','GD']]
    away_games = int_games[['away_team','home_team','tournament','neutral','recent','date','GD']]
    home_games['loc'] = 'H'
    away_games['loc'] = 'A'
    away_games['GD'] = -1 * away_games['GD']
    col_rename = ['team','opponent','tourney','neutral','recent','date','GD','location']
    away_games.columns = col_rename
    home_games.columns = col_rename
    all_games = pd.concat([away_games,home_games])
    all_games.loc[all_games['neutral'],'location'] = 'N'
    all_games.reset_index(inplace=True)
    relevant_games = all_games[all_games['date'].dt.year > 2014]

In [7]:
if pr_alo:
    select_df = int_games[['home_team','away_team','tournament','neutral','recent','date','GD']]
    col_renamev2 = ['team','opponent','tourney','neutral','recent','date','GD']
    select_df.columns = col_renamev2
    relevant_games = select_df[select_df['date'].dt.year>=2014]

In [8]:
def tourney_weights(t_name):
    # WC - 4
    # Continental Cup & Confederations Cup - 3
    three_list = ["AFC Asian Cup","AFC Challenge Cup","African Cup of Nation",
                  "CFU Caribbean Cup","CONCACAF Championship","Gold Cup","Oceania Nations Cup",
                  "UAFA Cup","UEFA Euro","Confederations Cup"]
    # WC Qualifying & CC Qualifying - 2.5
    two_list = ["FIFA World Cup qualification","AFC Asian Cup qualification","AFC Challenge Cup qualification",
                "African Cup of Nations qualification","CFU Caribbean Cup qualification",
                "CONCACAF Championship qualification","Gold Cup qualification",
                "Oceania Nations Cup qualification","UAFA Cup qualification","UEFA Euro qualification"]
    # all others 1
    if t_name == 'FIFA World Cup':
        return 4
    elif t_name in three_list: 
        return 3
    elif t_name in two_list:
        return 2.5
    else:
        return 1

In [9]:
relevant_games['weights'] = relevant_games['tourney'].apply(tourney_weights)

/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
relevant_games['GD_recent_weight'] = relevant_games['weights'] * relevant_games['GD'] * np.exp(-1*(relevant_games['recent'].dt.days)/np.max(np.absolute(relevant_games['recent'].dt.days)))

/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
gd_group = relevant_games.groupby(['team','opponent'])['GD_recent_weight'].agg('sum')

In [12]:
group_scores = gd_group.reset_index()

In [13]:
group_scores=group_scores.rename(columns = {'GD_recent_weight':'GD'})

In [14]:
test = group_scores[group_scores['GD'] == 0]
test['GD'] = -(0.5)
add_values = group_scores.append(test)
add_values.reset_index(inplace=True)

/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [15]:
add_values.loc[add_values['GD'] == 0,'GD'] = 0.5

In [16]:
def create_from(row):
    if row['GD']<0:
        return row['team']
    else:
        return row['opponent']
def create_to(row):
    if row['GD']>0:
        return row['team']
    else: 
        return row['opponent']

In [17]:
from_scores = add_values
from_scores['from'] = from_scores.apply(create_from, axis=1)
from_scores['to'] = from_scores.apply(create_to,axis=1)

In [18]:
from_scores['GD'] = np.absolute(from_scores['GD'])
final_graph_df = from_scores[['from','to','GD']]

In [19]:
final_graph_df.reset_index(inplace=True)

In [20]:
simple_group_df = final_graph_df.groupby(['from','to'])['GD'].agg('sum')

In [21]:
simple_df = simple_group_df.reset_index()

### Final Start Page Rank

In [22]:
import networkx as nx

In [23]:
G=nx.from_pandas_dataframe(simple_df, source='from', target='to', edge_attr='GD',create_using=nx.DiGraph())

In [24]:
w_ng = nx.pagerank(G)

In [25]:
sorted_w = sorted(w_ng.items(), key=lambda x: x[1],reverse=True)

In [26]:
w_df = pd.DataFrame.from_dict(w_ng.items())

In [27]:
w_df['scaled'] = (w_df[1] - w_df[1].mean())/(w_df[1].std())

In [28]:
w_df['rank'] = w_df['scaled'].rank(ascending=False)

## Compare to World Cup Teams

In [42]:
world_cup = pd.read_csv("https://raw.githubusercontent.com/lbenz730/world_cup/master/fixtures.csv")

In [64]:
teams1 = world_cup['team'].unique()
teams2 = world_cup['opponent'].unique()

In [68]:
joined_list = list(set([y for x in [teams1, teams2] for y in x]))

In [74]:
w_df[w_df[0].isin(joined_list)].sort_values('rank')

,0,1,scaled,rank
23,Germany,0.014541,3.148790,1.0
36,England,0.013149,2.710363,3.0
106,Brazil,0.013136,2.706048,4.0
94,Portugal,0.013126,2.702931,5.0
11,Argentina,0.012241,2.424359,6.0
60,France,0.012211,2.414840,7.0
156,Mexico,0.011735,2.264878,8.0
25,Spain,0.011689,2.250291,11.0
164,Colombia,0.011193,2.094014,12.0
32,Costa Rica,0.010769,1.960436,13.0
